In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import descarteslabs as dl
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import load_model
from tensorflow import keras
from tqdm.notebook import tqdm

from scripts import dl_utils
from scripts.viz_tools import band_descriptions, plot_image_grid

In [ ]:
def unit_norm(samples):
    means = [1298.7330617049158, 1046.728074661139, 968.1679384402669, 805.7169469984433, 1001.5350089770019, 1721.8117738966116, 2074.660173162641, 1906.273653848785, 2253.9860009889144, 295.4803929281597, 1544.334239964913, 900.2133779561017]
    deviations = [366.08860308693147, 378.157880474536, 389.7569162648193, 478.7348213604553, 435.20976171883257, 631.4499370533496, 798.0576089639353, 766.4725319841734, 892.286826414747, 131.88545750591763, 726.3113441205015, 601.8741576997301]
    normalized_samples = np.zeros_like(samples).astype('float32')
    for i in range(0, 12):
        #normalize each channel to global unit norm
        normalized_samples[:,:,:,:,i] = (np.array(samples)[:,:,:,:,i] - means[i]) / deviations[i]
    return normalized_samples

In [ ]:
model = load_model('../models/3DConv_2x48x48x12_temporal_patch.h5', custom_objects={'LeakyReLU': keras.layers.LeakyReLU,
                                                                                    'ELU': keras.layers.ELU,
                                                                                    'ReLU': keras.layers.ReLU
                                                                                   })

In [ ]:
START_DATE = '2020-01-01'
END_DATE = '2021-01-01'
MOSAIC_PERIOD = 3  # the period over which to mosaic image data in months
SPECTROGRAM_INTERVAL = 2  # For spectrogram analysis, the time from the start of one mosaic to the start of the next,
 # in number of mosaic periods
    
# Set rect width in pixels. Only required for point samples
num_pixels = 48
# Convert pixels to degrees. Heuristic, not geographically sound
# Better to go with slightly bigger patches that can then be cropped
rect_width = (num_pixels / 100) / 111.32

In [ ]:
coords = [116.60519642387457, -8.641674779337336]
coords = [-10.200799, 123.546910 ][::-1]
polygon = dl_utils.rect_from_point(coords, rect_width)

In [ ]:
pairs = []
try:
    mosaics, _ = dl_utils.download_mosaics(polygon, START_DATE, END_DATE, MOSAIC_PERIOD, method='min')
    new_pairs = dl_utils.pair(mosaics, SPECTROGRAM_INTERVAL)
    pairs += [p for p in new_pairs if dl_utils.masks_match(p)]
except:
    print('Failure', polygon)
print(len(pairs), "pairs of images extracted")

In [ ]:
resolution = model.input_shape[3]
cropped_pairs = []
for pair in pairs:
    a = []
    for image in pair:
        cropped = dl_utils.pad_patch(image.filled(0), resolution)
        a.append(cropped)
        #print(cropped.shape)
    cropped_pairs.append(a)

In [ ]:
plt.figure(figsize=(6, 3), dpi=150)
plt.subplot(1,2,1)
plt.title('Time 1')
plt.imshow(np.clip((unit_norm(cropped_pairs)[0][0][:,:,3:0:-1] + 1) / 2, 0,1))
plt.axis('off')
plt.subplot(1,2,2)
plt.title('Time 2')
plt.axis('off')
plt.imshow(np.clip((unit_norm(cropped_pairs)[0][1][:,:,3:0:-1] + 1) / 2, 0,1))
plt.show()

In [ ]:
print("Predictions:")
for pred in model.predict(unit_norm(cropped_pairs))[:,1]:
    print(f"{pred:.4f}")